In [2]:
#This file pulls in the data needed to make automated stock purchases based off of the previous buisiness day's info.
#It then sets a sale order.
#I had a goal of running this remotely and automatically so it also has an automated texting feature. 
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import json
import pprint as pprint
import pandas as pd
from joblib import dump, load
import requests
import math
import alpaca_trade_api as tradeapi
#Twilio is the service I use to text updates on info and events for when I hope to run code remotely and automticaly.
from twilio.rest import Client
account_sid = 'sid'
auth_token = 'token'
client = Client(account_sid, auth_token)
import os
base_url = 'https://paper-api.alpaca.markets'
api_key = 'APIKey'
secret_key = 'secretKey'
api = tradeapi.REST(api_key, secret_key,base_url)
account = api.get_account()
df = pd.read_csv('./v3100/V3_2StockList.csv')
tckr = df['Tckr'].tolist()
#pp=Purchasing Power
pp = float(round(math.floor((round(float(account.cash),1)/3)*.97),2))
pp


27666.0

In [6]:
#The first step to a days trading process is collecting the Open, Close, High and Low for the stocks in our selected group. 
#I collect the previous days info so that the model can be tested on its acuracy in predicting todays high. If the accuracy is below
#.9985 I dont make any actual purchases but instead let the program purchase 3 on my paper account for data collection purposes. 
price_info= []
count = 0
for i in tckr:
    try:
        barset = api.get_barset(i, 'day', limit=2)
        
        openingY = barset[i][0].o
        closingY = barset[i][0].c
        highY = barset[i][0].h
        lowY = barset[i][0].l
        openingT = barset[i][1].o
        closingT = barset[i][1].c
        highT = barset[i][1].h
        lowT = barset[i][1].l
        info = {'Ticker':f'{i}',
                'openingY':openingY,
                'closingY':closingY,
                'highY':highY,
                'lowY':lowY,
                'openingT':openingT,
                'closingT':closingT,
                'highT':highT,
                'lowT':lowT}
        pprint(i)
        count+=1
        price_info.append(info)
#         pprint(price_info)
        
        

    except:
        
        count+=1

        print(f'result error{i}')
        pass
df = pd.DataFrame(price_info)

df
  

'RKLY'
'POLA'
'IPDN'
'LKCO'
'OXBR'
'VVPR'
'BROS'
'VRPX'
'MARA'
'WAFU'
'ORPH'
'DBGI'
'EVK'
'BRQS'
'HX'
'IZEA'
'AHT'
'USWS'
'SOS'
'PRPO'
'EYES'
'OCGN'
'CIDM'
'INDP'
'PPSI'
'AIHS'
'WPG'
'AMC'
'XELA'
'WORX'
'MTC'
'EXPR'
'LGHL'
'AMTX'
'NAOV'
'RCON'
'LEDS'
'CARV'
'CREX'
'MRIN'
'GOTU'
'NXTP'
'AMR'
'CLSK'
'OEG'
'SINO'
'CRTD'
'ENG'
'SAVA'
'ARTL'


,Ticker,openingY,closingY,highY,lowY,openingT,closingT,highT,lowT
0,RKLY,4.4700,4.030000,4.5400,4.020000,4.0600,4.10000,4.180000,4.0000
1,POLA,3.1300,3.110000,3.2498,3.080000,3.0800,3.04000,3.220000,2.9300
2,IPDN,1.0100,0.975100,1.0100,0.975000,1.0100,1.02000,1.050000,0.9649
3,LKCO,0.7895,0.729600,0.8230,0.720000,0.7600,0.70030,0.780000,0.6810
4,OXBR,5.9800,5.780000,6.0000,5.650000,5.7900,5.68000,5.800000,5.5100
5,VVPR,2.6500,2.650000,2.7900,2.536200,2.5600,2.47000,2.570000,2.4000
6,BROS,52.9400,50.030000,53.8900,50.030000,50.8000,50.80000,51.870000,48.7700
7,VRPX,2.2400,2.130000,2.2500,2.085000,2.1600,2.11230,2.190000,2.0850
8,MARA,27.1400,25.715000,27.8797,25.500000,25.6214,23.90000,25.760000,23.8300
9,WAFU,3.2500,3.130000,3.2550,2.920000,3.1000,2.90500,3.190000,2.8200


In [26]:
#Each time the program runes it updates its education with yesterdays data. After a few months of experimenting with this 
#I have not seen any decay in the effectivness of the models. 
X = df[['openingY', 'closingY', 'highY', 'lowY']]
y = df["highT"].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
dump(X_scaler, './v3100/V3_2X_scaler.joblib') 
dump(y_scaler, './v3100/V3_2Y_scaler.joblib') 
model = LinearRegression()
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
model.fit(X_train_scaled, y_train_scaled)
predictions = model.predict(X_test_scaled)
r2 = model.score(X_test_scaled, y_test_scaled)
dump(model, './v3100V.3_2.joblib') 
ldf = df[['openingT', 'closingT', 'highT', 'lowT']]
sc_new_in = X_scaler.transform(ldf)
new_out = model.predict(sc_new_in)
new_out = y_scaler.inverse_transform(new_out)
df["next_high"]= new_out
df["change"]= df['next_high']/df["closingT"]*100-100
rdf = df.sort_values(by=['change'], axis=0, ignore_index=True, inplace=False, ascending = False)
rdf = rdf.drop(0, axis=0)
#This is where I choose the top 3 when ranked by projected change. I also have the purchasing power split into thirds 
#in the first section of this file. 
purchase_list = rdf.head(3)
#I use the HighT to devide my PP so as to make sure it does not over spend when buying stocks. This is also the reason for using .97 of tthe total paper funds.
share_list = round(pp/purchase_list.highT)
c = 1
message_list = []
for i in purchase_list.Ticker:
    message_list.append(purchase_list.Ticker[c])
    message_list.append(share_list[c])
    message_list.append(purchase_list.change[c])
    api.submit_order(
        symbol=i,
        qty=share_list[c],
        side='buy',
        type='market',
        time_in_force='day'
    )
    c+=1
message = client.messages \
                .create(
                    body=f"Model accuracy rating of {r2}. {message_list[1]} Shares of {message_list[0]} projected with {message_list[2]}change. {message_list[4]} Shares of {message_list[3]} projected with {message_list[5]}change. {message_list[7]} Shares of {message_list[6]} projected with {message_list[8]}change. ",
                    from_='+19703153376',
                    to='my-number'
                )



In [31]:
# This pulls in the positions(stocks successfully purchased)
positions = api.list_positions()
#This sets a stopping point for the stocks. 
if len(positions)>0:
    c = 0
    for i in positions:
        api.submit_order(
        symbol=positions[c].symbol,
        qty=positions[c].qty,
        side='sell',
        type='trailing_stop',
        trail_percent=1.5,
        time_in_force='gtc',
        )
        c+=1
    print("Short sales set")
else:
    print("No positions held")

No positions held
